In [2]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics, preprocessing
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from catboost import Pool, CatBoostClassifier
import lightgbm as lgb


In [3]:
#Importing
train_data = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/training_v2.csv')
test_data =  pd.read_csv('/Users/s0c02nj/Desktop/WiDS/unlabeled.csv')

sub_data = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/samplesubmission.csv')
data_dict = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/WiDS Datathon 2020 Dictionary.csv')

In [4]:
def weighted_classt(x): 
    if pd.isna(x):
        return np.nan
    elif x < 15: 
        return 'very severely underweight' 
    elif x >= 15 and x < 16: 
        return 'severely weight' 
    elif x >=16 and x < 18.5: 
        return 'underweight' 
    elif x >= 18.5 and x < 25: 
        return 'healthy weight' 
    elif x >= 25 and x < 30: 
        return 'overweight'
    elif x >= 30 and x < 35: 
        return 'class 1' 
    elif x >= 35 and x < 40: 
        return 'class 2' 
    else: 
        return 'class 3' 

In [5]:
train_data['weightclass'] = train_data['bmi'].map(weighted_classt)
test_data['weightclass'] = test_data['bmi'].map(weighted_classt)

In [6]:
cat_var = list(data_dict[data_dict['Data Type'].isin(['binary','string','integer'])]['Variable Name'])
cat_var.remove('icu_admit_type')
cat_var.remove('bmi')
cat_var = cat_var[1:]

In [ ]:
cont_var = list(data_dict[data_dict['Data Type'].isin(['numeric'])]['Variable Name'])
cont_var.remove('pred')
cont_var = cont_var + ['bmi']

#### Baseline

In [ ]:
x_train = train_data.drop('hospital_death',axis=1)
y_train = train_data['hospital_death']

In [ ]:
x_test = test_data.drop('hospital_death',axis=1)

In [ ]:
x_comb = pd.concat([x_train,x_test],sort=False)

In [ ]:
### impute bmi values based on height and weight
count = 0
bmi_list = []
for i in tqdm(range(x_comb.shape[0])):
    
    if pd.isnull(x_comb["weight"].iloc[i]) or pd.isnull(x_comb["height"].iloc[i]):
        bmi_list.append(x_comb["bmi"].iloc[i])
        continue
    else:
        
        count += 1
        bmi_list.append(x_comb["weight"].iloc[i] / ((x_comb["height"].iloc[i]/100) ** 2))

In [ ]:
x_comb['bmi'] = bmi_list

In [ ]:
x_comb["apache_4a_hospital_death_prob"] = x_comb["apache_4a_hospital_death_prob"].replace(-1, np.nan)
x_comb["apache_4a_icu_death_prob"] = x_comb["apache_4a_icu_death_prob"].replace(-1, np.nan)

In [ ]:
new_cat_var = list(set(cat_var).intersection(set(x_comb.columns)))

In [ ]:
cat_var2 = list(set(cat_var2).intersection(set(x_comb.columns)))

In [ ]:
for col in tqdm(cat_var2):
    le = LabelEncoder()
    x_comb[col] = le.fit_transform(x_comb[col].astype(str))

In [ ]:
new_cont_var = list(set(cont_var).intersection(set(x_comb.columns)))

In [ ]:
x_comb["age"].mean()

In [ ]:
# for col in tqdm(new_cont_var):
#     x_comb[col] = x_comb[col].fillna(x_comb[col].mean())

In [ ]:
count_var = []
for i,col in tqdm(enumerate(new_cat_var)):
    counter = Counter(x_comb[col])
    x_comb[str(col)+'count'] = x_comb[col].apply(lambda x:counter[x])
    count_var.append(str(col)+'count')

In [ ]:
age_cat = []

for i in tqdm(range(0,len(x_comb))):
    
    val = x_comb['age'].iloc[i]
    
    if val >= 15 and val <= 24: 
        age_cat.append('igen')
    
    elif val >= 25 and val <= 54: 
       age_cat.append('Prime_working_Age')
    
    elif val >= 55 and val <= 64: 
        age_cat.append('Mature_working_Age')
        
    else: 
        age_cat.append('Elderly_working_Age')


In [ ]:
x_comb['age_category'] = age_cat
le = LabelEncoder()
x_comb['age_category'] = le.fit_transform(x_comb['age_category'].astype(str))

In [ ]:
#List to be biined -----> VITALS
to_binned_columns = list(data_dict[data_dict['Category'] == 'vitals']['Variable Name'].unique())

In [ ]:
#inter_cols3
#x_comb['d1_heartrate_max_cat'] = 

binned_cols = []

for cols in tqdm(to_binned_columns) :
    x_comb[cols+'_binned'] = list(pd.cut(x_comb[cols], 10,labels=False))
    binned_cols.append(cols+'_binned')
    

In [ ]:
x_comb['height'+'_binned'] = list(pd.cut(x_comb['height'], 8,labels=False))

In [ ]:
#binned_cols

In [ ]:
#train_data.columns[0:100]

list_imp_cols = ['icu_type','weightclass','age_category','gender','d1_heartrate_max_binned','height_binned']

inter_cols4 = []

for col1 in tqdm(list_imp_cols) :
    for col2 in list_imp_cols:
        if col1 !=col2 :
            x_comb[col1+col2] = x_comb[col1].astype(str) + '_' + x_comb[col2].astype(str)
            inter_cols4.append(col1+col2)
            

In [ ]:
for cols in tqdm(inter_cols4):
    le = LabelEncoder()
    x_comb[cols] = le.fit_transform(x_comb[cols].astype(str))

In [ ]:
#train_data.columns
# list_groups = list(data_dict['Category'].unique())
# list_groups = list_groups[1:]

# grp_ls = []

# for grp in tqdm(list_groups):
    
#     grp_ls.append(list(data_dict[data_dict['Category'] == grp]['Variable Name']))

In [ ]:
# inter_cols5 = []

# for i in tqdm(range(0,len(grp_ls))):
#     p = grp_ls[i]
#     cols = list(set(p).intersection(set(x_comb.columns)))
#     #str_cols = '_'.join(cols)
    
#     x_comb['grp_cols'+str(i)] = x_comb[cols].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)
#     inter_cols5.append('grp_cols'+str(i))


In [ ]:
# for cols in tqdm(inter_cols5):
#     le = LabelEncoder()
#     x_comb[cols] = le.fit_transform(x_comb[cols].astype(str))

In [ ]:
#2way interaction
bias_cols = ['gender','age']

In [ ]:
inter_cols6 = []

for i in tqdm(bias_cols):
    
    for j in binned_cols :
        
        x_comb[i+'_'+j] = x_comb[i].astype(str) + '_' + x_comb[j].astype(str)
        
        inter_cols6.append(i+'_'+j)

In [ ]:
#Critical -3way interaction

# inter_cols6 = []

x_comb['age'+'gender'+'weighted_classt'] = x_comb['age'].astype(str) + '_' + x_comb['gender'].astype(str) + '_' + x_comb['weightclass'].astype(str)
x_comb['age'+'gender'+'ethnicity'] = x_comb['age'].astype(str) + '_' + x_comb['gender'].astype(str) + '_' + x_comb['ethnicity'].astype(str)

inter_cols6 = inter_cols6 + ['age'+'gender'+'weighted_classt','age'+'gender'+'ethnicity']


In [ ]:
#inter_cols6

In [ ]:
for cols in tqdm(inter_cols6):
    le = LabelEncoder()
    x_comb[cols] = le.fit_transform(x_comb[cols].astype(str))

In [ ]:
#final_cat_var = new_cat_var + count_var + ['age_category'] + inter_cols1 + inter_cols2 + inter_cols3 + inter_cols4

##### Target Encoding

In [ ]:
temp_train = x_train.copy()
temp_train['hospital_death'] = train_data['hospital_death']

In [ ]:
cat_count_var = []

for i,col in tqdm(enumerate(new_cat_var)):
    mean = temp_train['hospital_death'].mean()
    
    #Compute the number of values and the mean of each group
    agg = temp_train.groupby(col)['hospital_death'].agg(['count', 'mean'])
    counts = agg['count']
    means =  agg['mean']
    
    #Compute the "smoothed" means
    m= 3
    smooth = (counts * means + m * mean) / (counts + m)
    
    #Final_val
    x_comb[str(col)+'count_new'] = x_comb[col].map(smooth)
    cat_count_var.append(str(col)+'count_new')
    

In [ ]:
x_comb.shape

In [ ]:
# # Remove Features with more than 70 percent missing values
data_missing = (x_comb.isnull().sum() / len(x_comb)).sort_values(ascending = False)
data_missing = data_missing.index[data_missing > 0.9]

x_comb = x_comb.drop(columns = data_missing)

In [ ]:
cat_count_var_new = list(set(cat_count_var).intersection(x_comb.columns))
#x_comb.isna().sum()

In [ ]:
for col in tqdm(cat_count_var_new):
    le = LabelEncoder()
    x_comb[col] = le.fit_transform(x_comb[col].astype(str))

In [ ]:
aggs = {}

for col in tqdm(new_cat_var):
    aggs[col] = ['nunique','count']
    

In [ ]:
new_cat_var = list(set(new_cat_var).intersection(set(x_comb.columns)))

In [ ]:
#new_cat_var

In [ ]:
for col in tqdm(new_cat_var):
    
    aggs_temp = aggs.copy()
    aggs_temp.pop(col)
    
    agg_df = x_comb.groupby(col).agg(aggs_temp).reset_index()
    agg_df.columns = [col] + [col + '_' + c[0] +'_' + c[1] for c in agg_df.columns[1:]]
    
    x_comb = pd.merge(left = x_comb, right= agg_df, how='left',
                    left_on=[col], right_on=[col])
    

In [ ]:
#x_comb.to_csv('/Users/s0c02nj/Desktop/WiDS/file_.csv')

In [ ]:
x_comb['hospital_icu_id'] = x_comb['hospital_id'].astype(str) + '_' +  x_comb['icu_id'].astype(str)

le = LabelEncoder()
x_comb['hospital_icu_id'] = le.fit_transform(x_comb['hospital_icu_id'].astype(str))

In [ ]:
x_comb = x_comb.drop(['encounter_id','patient_id'],axis=1)

In [ ]:
train_x = x_comb[0:91713]
test_x = x_comb[91713:]

In [ ]:
x_comb.shape

In [ ]:
x1_train, x1_val, y1_train, y1_val = train_test_split(train_x, y_train, 
                                                      test_size=0.2,
                                                      random_state=0,
                                                      stratify = y_train)

In [ ]:
random_state = 42
np.random.seed(random_state)

lgb_params = {
    'boost': 'gbdt',
    'learning_rate': 0.01,
    'max_depth':8,
    'metric':{'auc'},
    'num_threads': -1,
    'objective': 'binary',
    'verbosity': 1,
    'num_leaves':16,
    'bagging_fraction':0.5, #rows
    'bagging_frequency':5,
    'min_data_in_leaf':50,
    'feature_fraction':0.25, # features
    'lambda_l1':10,
    'lambda_l2':15,
    'bagging_seed':2019,
    'min_gain_split':0.3
}

In [ ]:
trn_data = lgb.Dataset(x1_train, label= y1_train)
val_data = lgb.Dataset(x1_val,  label=  y1_val)

In [ ]:
lgb_clf = lgb.train(lgb_params,
                    trn_data,
                    13000,
                    valid_sets = [trn_data, val_data],
                    early_stopping_rounds=2000,
                    verbose_eval=500
                    )

In [ ]:
pred = lgb_clf.predict(test_x)

In [ ]:
data_sub = x_test[['encounter_id']]
data_sub['hospital_death'] = pred

In [ ]:
data_sub.to_csv('/Users/s0c02nj/Desktop/WiDS/sub_lgb_keeping_missing_as_itis_type2_9th_feb_cv1.csv',index = False)

In [ ]:
fig, ax = plt.subplots(figsize=(12,30))
lgb.plot_importance(lgb_clf, max_num_features=180, height=0.8, ax=ax)
ax.grid(False)
plt.title("LightGBM - Feature Importance", fontsize=12)
plt.show()

In [ ]:
kf = StratifiedKFold(n_splits=10, random_state=42, shuffle=True)

In [ ]:
predictions = np.zeros(len(x_test))
num_models = 0

for train_index, valid_index in tqdm(kf.split(train_x, y_train)):
    
    d_train = lgb.Dataset(train_x.iloc[train_index], label=y_train[train_index])
    d_val = lgb.Dataset(train_x.iloc[valid_index], label=y_train[valid_index])

    clf = lgb.train(lgb_params, d_train, 130000, verbose_eval=1000, 
                    valid_sets = [d_train, d_val], early_stopping_rounds = 3000)
    

    
    num_models += 1
    
    predictions += clf.predict(test_x)

In [ ]:
predictions_10 = predictions / 10.0

In [ ]:
data_sub = x_test[['encounter_id']]

In [ ]:
data_sub['hospital_death'] = predictions_10

In [ ]:
data_sub.to_csv('/Users/s0c02nj/Desktop/WiDS/sub_10fold_keep_missing_as_it_is_9thFeb2020_lgb.csv_type2.csv',index = False)



In [ ]:
sub['hospital_death'] = pred_ens

In [ ]:
sub.to_csv('/Users/s0c02nj/Desktop/WiDS/sub_ensemble_9thFeb2020.csv',index = False)

In [ ]:
sub = pd.DataFrame()
sub['encounter_id'] = a1['encounter_id']

In [ ]:
pred_ens = (
            a1['hospital_death'] + a2['hospital_death'] 
           )/2.0

In [ ]:
a1 =  pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub_10fold_keep_missing_as_it_is_9thFeb2020_lgb.csv')
a2 =  pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub_10fold_keep_missing_as_it_is_9thFeb2020_lgb.csv')
# a3 =  pd.read_csv('/Users/s0c02nj/Desktop/WiDS/Submission/sub_ensemble_gm1.csv')
# a4 =  pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub_ensemble_gm2.csv')
# a5 =  pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub8_lgb_10fold1.csv')
# a6 =  pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub8_lgb_10fold.csv')
# a7 =  pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub10_lgb_kfold2.csv')
# a8 =  pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub5_lgb_ensemble12_fold_01022020.csv')
# a9 =  pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub11_ensemble_imp1.csv')
# a10 = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub12_catb.csv')
# a11 = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub3_lgb_ensemble1_01022020.csv')
# a12 = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub4_ensemble1_01022020.csv')
# a13 = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub5_lgb_ensemble12_fold_01022020.csv')
# a14 = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub3_10fold_4thFeb2020.csv')
# a15 = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub_ensemble_gm3.csv')
# a16 = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub6_10fold_4thFeb2020.csv')
# a17 = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub60_10fold_4thFeb2020.csv')
# a18 = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub2_ensemble_5thFeb2020.csv')
# a19 = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub_10fold_5thFeb2020_type2.csv')

In [ ]:
#train_x['encounter_id'].unique().shape
sub['hospital_death'] = pred_ens

In [ ]:
#sub_data

In [ ]:
# data_sub = x_test[['encounter_id']]
# data_sub['hospital_death'] = preds

In [ ]:
#x_test.head()

In [ ]:
#ls

In [ ]:
#data_sub.to_csv('/Users/s0c02nj/Desktop/WiDS/sub2.csv',index = False)

In [ ]:
#data_sub.shape